# Experiment Notebook

This notebook documents **experimental work** conducted during the development of the **Elderly Monitoring AI System**.

⚠️ **Important**  
- This notebook explores an **experimental cloud-based approach** to stranger detection.  
- It is **NOT part of the final runtime system**.  
- The runnable, offline pipeline is located in the `/src` directory.


# Stranger Detection Experiment (Cloud-Based Variant)

This experiment explores a **cloud-synchronized stranger detection approach**, where facial encodings are stored in a remote PostgreSQL database.

This design was evaluated during development but **was not selected** for the final deployed system due to privacy, latency, and dependency considerations.


## Purpose

- Detect human faces from live webcam input
- Compare detected faces against a **remote database of known face encodings**
- Label unknown faces as *Stranger*


## Environment

- Python 3.10
- face_recognition (dlib-based)
- OpenCV
- psycopg2 (PostgreSQL client)

See `requirements-experiments.txt` for the full dependency list.


## Database Notes

- Backend: PostgreSQL (AWS RDS during experimentation)
- Table: `known_faces`
- Columns:
  - `id` (integer)
  - `name` (text)
  - `encodings` (bytea – pickled 128D vectors)
  - `created_at` (timestamp)

⚠️ **Credentials are intentionally omitted**.  
Set database credentials locally before running this notebook.


In [ ]:
import face_recognition
import cv2
import pickle
import psycopg2
import numpy as np
import matplotlib.pyplot as plt

print("✅ Libraries imported")

In [ ]:
# Database configuration (EXAMPLE ONLY)
# Replace with local credentials before running

DB_CONFIG = {
    "host": "<AWS_RDS_ENDPOINT>",
    "database": "<DATABASE_NAME>",
    "user": "<USERNAME>",
    "password": "<PASSWORD>",
    "port": "5432"
}

print("⚠️ Database configuration placeholders loaded")

In [ ]:
class StrangerDetectionSystem:
    def __init__(self, db_config, model='hog', tolerance=0.45):
        self.db_config = db_config
        self.model = model
        self.tolerance = tolerance
        self.known_encodings = []
        self.known_names = []
        
        # Load encodings from the database
        self.sync_from_cloud()

    def sync_from_cloud(self):
        """Download face encodings from PostgreSQL."""
        try:
            conn = psycopg2.connect(**self.db_config)
            cur = conn.cursor()
            cur.execute("SELECT name, encodings FROM known_faces;")
            
            self.known_encodings = []
            self.known_names = []

            for name, blob in cur.fetchall():
                self.known_names.append(name)
                self.known_encodings.append(pickle.loads(blob))

            cur.close()
            conn.close()
            print(f"✅ Synced {len(self.known_names)} identities from database")
        except Exception as e:
            print(f"❌ Database sync failed: {e}")

    def recognize_faces(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        locations = face_recognition.face_locations(rgb_frame, model=self.model)
        encodings = face_recognition.face_encodings(rgb_frame, locations)

        for (top, right, bottom, left), encoding in zip(locations, encodings):
            matches = face_recognition.compare_faces(
                self.known_encodings, encoding, tolerance=self.tolerance
            )
            name = "Stranger"

            if True in matches:
                match_index = matches.index(True)
                name = self.known_names[match_index]

            color = (0, 255, 0) if name != "Stranger" else (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.putText(frame, name, (left, top - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        return frame

In [ ]:
# Initialize system (requires valid DB credentials)
system = StrangerDetectionSystem(DB_CONFIG)

In [ ]:
# Start webcam stream
cap = cv2.VideoCapture(0)
print("Starting live stream — press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = system.recognize_faces(frame)
    cv2.imshow("Stranger Detection (Experimental)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Stream stopped")

## Observations & Notes

- Known faces are correctly retrieved from the database when credentials are valid.
- Unknown faces are labeled as **Stranger** and highlighted in red.
- Network latency and database availability affect responsiveness.
- This experiment informed the decision to use **local encodings** in the final system.
